In [1]:
import torch as T
import numpy as np
import gym

from IPython.display import clear_output
from Ractor_critic import ActorCritic

In [2]:
from enviroment_class import CustomEnv

env = CustomEnv(obstacle_turn = True,
                vizualaze     = True,
                Total_war     = True,
                inp_dim       = 500,
                head_velocity = 0.001,#0.005
                num_obs       = 5,
                num_enemy     = 1,
                size_obs      = [50, 60],
                rew_col       = -70,
                rew_win       = 100,
                rew_defeat    = -100,
                steps_limit   = 1000,
                EnemyLidSet   = [45,90],
                AllyLidSet    = [40, 90])
_ = env.reset()


pygame 2.1.0 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/parkhomenko/PycharmProjects/RL/venv/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [3]:
agent = ActorCritic(lr = 0.0005, 
                    input_conv_dims = (15,3),
                    allias_state = 5, 
                    enemy_state = 5, 
                    n_actions  = env.action_space.n,
                    cuda = 'cuda:0',
                    gamma = 0.96)

/home/parkhomenko/PycharmProjects/RL/venv/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [4]:
scores = []
save_score = -np.inf

n_steps = 5000
T_MAX = 20

In [ ]:
for i in range(n_steps):
    observation = env.reset()
    
    done = False
    ep_steps = 0
    r = 0
    hx, cx = T.zeros(64).to(agent.actor.device), T.zeros(64).to(agent.actor.device)

    while not done:

        action, hx, cx = agent.choose_action(observation, hx, cx)
        observation, reward, done, info = env.step(action)
        agent.store_mem(reward)
        
        r += reward
        ep_steps+=1
        
        if ep_steps % T_MAX == 0 or done:
            hx, cx = hx.detach(), cx.detach()
            loss = agent.learn(observation, done)
            
    scores.append(r)
    
    avg_score = np.mean(scores[-100:])
    if avg_score > save_score:
        agent.save_models()
        save_score = avg_score
    
    if not bool(i%10):clear_output()
    print(i,' reward:', r,' avg score:', avg_score,' loss:', loss)

0  reward: -570.4774890452662  avg score: -570.4774890452662  loss: (-750.4937744140625, 3938.723388671875)
.... saving models ....
1  reward: -194.0879565232863  avg score: -382.28272278427625  loss: (-1704.266845703125, 3855.711669921875)
.... saving models ....
2  reward: -290.418617716693  avg score: -351.6613544284152  loss: (-133.16024780273438, 4383.40625)
.... saving models ....
3  reward: -32.06923008591918  avg score: -271.7633233427912  loss: (-1326.242919921875, 1868.22314453125)
.... saving models ....
4  reward: 251.63586693481358  avg score: -167.08348528727024  loss: (-460.58819580078125, 3003.79150390625)
.... saving models ....
5  reward: -83.26474228376397  avg score: -153.11369478668587  loss: (-1793.795166015625, 2486.399658203125)
.... saving models ....
6  reward: -140.62449335540867  avg score: -151.32952315364628  loss: (-808.1724243164062, 3494.767333984375)
.... saving models ....
7  reward: 78.46101391626533  avg score: -122.6057060199073  loss: (-1184.86450

In [ ]:
# agent.load_models()

In [ ]:
import matplotlib.pyplot as plt
def plot_learning_curve(scores, x, figure_file):
    # loss_avg = np.zeros(len(scores))
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
        # loss_avg[i] = np.mean(loss[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    # plt.plot(x, loss_avg)
    plt.title('Average of previos 100 games')
    plt.savefig(figure_file)

In [ ]:
steps_list = [z for z in range(n_steps)]

plot_learning_curve(scores, steps_list, 'A3C_pong_final.png')

In [ ]:
from enviroment_class import CustomEnv

env = CustomEnv(obstacle_turn = True,
                vizualaze     = False,
                Total_war     = True,
                inp_dim       = 500,
                head_velocity = 0.0005,#0.005
                num_obs       = 5,
                num_enemy     = 1,
                size_obs      = [50, 60],
                rew_col       = -70,
                rew_win       = 100,
                rew_defeat    = -100,
                steps_limit   = 1000,
                EnemyLidSet   = [45,90],
                AllyLidSet    = [40, 90])
_ = env.reset()


env.get_statistic( model = agent, num_games = 1000)